Mock Data

In [1]:
import pandas as pd

data = {
    "market": ["India", "Brazil", "Germany", "Indonesia", "Mexico"],
    "market_size": [9, 7, 6, 8, 7],
    "digital_adoption": [8, 7, 9, 7, 6],
    "regulation": [6, 5, 8, 6, 6],
    "competition": [7, 6, 8, 5, 6]
}

df = pd.DataFrame(data)
df

,market,market_size,digital_adoption,regulation,competition
0,India,9,8,6,7
1,Brazil,7,7,5,6
2,Germany,6,9,8,8
3,Indonesia,8,7,6,5
4,Mexico,7,6,6,6


Define user-adjustable weights ----- Normalizing weights for each feature

In [2]:
weights = {
    "market_size": 0.35,
    "digital_adoption": 0.30,
    "regulation": 0.20,
    "competition": 0.15
}

# Normalize weights so they sum to 1
total = sum(weights.values())
weights = {k: v / total for k, v in weights.items()}

weights


{'market_size': 0.35,
 'digital_adoption': 0.3,
 'regulation': 0.2,
 'competition': 0.15}

Normalize data -- to make values for different features in that sample comparable

In [3]:
cols = ["market_size", "digital_adoption", "regulation", "competition"]

normalized = (df[cols] - df[cols].min()) / (df[cols].max() - df[cols].min())
normalized


,market_size,digital_adoption,regulation,competition
0,1.000000,0.666667,0.333333,0.666667
1,0.333333,0.333333,0.000000,0.333333
2,0.000000,1.000000,1.000000,1.000000
3,0.666667,0.333333,0.333333,0.000000
4,0.333333,0.000000,0.333333,0.333333


Invert competition---This is the business logic insight.

In [4]:
normalized["competition"] = 1 - normalized["competition"]
normalized


,market_size,digital_adoption,regulation,competition
0,1.000000,0.666667,0.333333,0.333333
1,0.333333,0.333333,0.000000,0.666667
2,0.000000,1.000000,1.000000,0.000000
3,0.666667,0.333333,0.333333,1.000000
4,0.333333,0.000000,0.333333,0.666667


Compute scores

In [5]:
df["score"] = sum(normalized[c] * weights[c] for c in cols)
df

,market,market_size,digital_adoption,regulation,competition,score
0,India,9,8,6,7,0.666667
1,Brazil,7,7,5,6,0.316667
2,Germany,6,9,8,8,0.500000
3,Indonesia,8,7,6,5,0.550000
4,Mexico,7,6,6,6,0.283333


Rank markets

In [6]:
ranked = df.sort_values("score", ascending=False)
ranked[["market", "score"]]

,market,score
0,India,0.666667
3,Indonesia,0.550000
2,Germany,0.500000
1,Brazil,0.316667
4,Mexico,0.283333


Generate explanations

In [7]:
def explain(row):
    return (
        f"{row['market']} ranks highly due to a strong balance of market size, "
        f"digital adoption, and regulatory environment, "
        f"with manageable competition given the selected priorities."
    )

ranked["explanation"] = ranked.apply(explain, axis=1)
ranked[["market", "score", "explanation"]]


,market,score,explanation
0,India,0.666667,India ranks highly due to a strong balance of ...
3,Indonesia,0.550000,Indonesia ranks highly due to a strong balance...
2,Germany,0.500000,Germany ranks highly due to a strong balance o...
1,Brazil,0.316667,Brazil ranks highly due to a strong balance of...
4,Mexico,0.283333,Mexico ranks highly due to a strong balance of...


Final recommendation

In [8]:
top = ranked.iloc[0]

print("Recommended Market:")
print(top["market"])
print("\nReason:")
print(top["explanation"])


Recommended Market:
India

Reason:
India ranks highly due to a strong balance of market size, digital adoption, and regulatory environment, with manageable competition given the selected priorities.
